### Build env for this notebook (temp. fix):

```
conda env create -n dnn_cpu python=3.12 numpy pandas scikit-learn tensorflow-cpu
conda activate dnn_cpu
```

In [28]:
import pickle
with open("../data/tmp_data/ml_data_at_sugaramino.pickle", "rb") as handle:
    ml_datasets = pickle.load(handle)

In [29]:
feature_name, X, y, sample_names, feature_names = ml_datasets[12]
feature_name

'3Di_KMER2'

In [30]:
X.shape

(54, 400)

In [31]:
from tensorflow import keras


def create_model(input_shape):
    # TODO possibly more layers, try without dropout
    model = keras.Sequential(
        [
            keras.layers.Input(shape=(input_shape,)),
            keras.layers.Dense(512, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(256, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dense(1, activation="sigmoid"),
        ]
    )

    model.compile(
        optimizer="adam",  # Adam(learning_rate=0.001)
        loss="binary_crossentropy",
        metrics=["Precision", "Recall", "AUC"],
    )  # ['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])
    #     metrics=['Precision', 'Recall', 'AUC']

    # TODO class weights
    # TODO early stopping?
    # TODO macro
    return model


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
# from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (
    cross_val_score,
    RepeatedStratifiedKFold,
    StratifiedKFold,
)
from sklearn.metrics import f1_score, balanced_accuracy_score
def nested_crossval(feature_name, X, y, sample_names, feature_names):
    np.random.seed(0)
    tf.random.set_seed(0)
    print(f"=== {feature_name} ===")
    # model = make_pipeline(VarianceThreshold(), StandardScaler(), SelectKBest(), SVC())
    preprocess = make_pipeline(VarianceThreshold(0.0), StandardScaler())

    history_logs = []
    scores = []

    # scale : 1 / (n_features * X.var()). 
    # larger variance and more features leads to a smoother decision boundary, 
    # where each sample has less influence
    splits=5
    repeats=10
    cross_validator = RepeatedStratifiedKFold(n_splits=splits, n_repeats=repeats)
    for fold_count, (train_idx, val_idx) in enumerate(cross_validator.split(X, y)):
        print(f"Fold {fold_count+1} out of {splits*repeats}")

        X_train, X_test = X[train_idx], X[val_idx]
        y_train, y_test = y[train_idx], y[val_idx]

        X_train = preprocess.fit_transform(X_train, y_train)
        X_test = preprocess.transform(X_test)

        model = create_model(X_train.shape[1])

        # TODO batch size epochs
        history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=0)
        # TODO balanced_accuracy for direct compare to SVM
        # TODO macro averaged?? maybe use sklearn score
        res = model.evaluate(X_test, y_test)
        print(res)
        # score = model.score(X_test, y_test)
        scores.append(res)
        # history_logs.append(history)
        # print(res)

    return scores


In [34]:
scores = nested_crossval(feature_name, X, y, sample_names, feature_names)

=== 3Di_KMER2 ===
Fold 1 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 8.6931e-04
[0.0008693148847669363, 1.0, 1.0, 1.0]
Fold 2 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 1.0324e-04
[0.00010323515743948519, 1.0, 1.0, 1.0]
Fold 3 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 0.0324
[0.03236909955739975, 1.0, 1.0, 1.0]
Fold 4 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 0.0209
[0.020859450101852417, 1.0, 1.0, 1.0]
Fold 5 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 0.0076
[0.007567738648504019, 1.0, 1.0, 1.0]
Fold 6 out of 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - AUC: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - loss: 0.0022
[0.002159623894840479, 1.0, 1.0, 1.0]
Fold 7 out of 50
1

In [35]:
scores = np.array(scores)
print(scores.shape)
print(
    f"Loss {scores[:,0].mean():.2f}+-{scores[:,0].std():.2f}, Precision {scores[:,1].mean():.2f}+-{scores[:,1].std():.2f} Recall {scores[:,2].mean():.2f}+-{scores[:,2].std():.2f}, AUC {scores[:,3].mean():.2f}+-{scores[:,3].std():.2f}"
)

(50, 4)
Loss 0.14+-0.22, Precision 0.96+-0.07 Recall 0.97+-0.07, AUC 0.99+-0.03


In [ ]:
# from sklearn.utils import class_weight
# import numpy as np

# class_weights = class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weights_dict = dict(enumerate(class_weights))

# model.fit(X_train, y_train, 
#           epochs=20, 
#           batch_size=32, 
#           class_weight=class_weights_dict, 
#           validation_data=(X_val, y_val))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(0)
tf.random.set_seed(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Input(shape=(len(feature_names),)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["Accuracy","Precision", "Recall", "AUC"])

history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

res = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {test_acc:.4f}")
print(res)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - AUC: 0.5263 - Accuracy: 0.4412 - Precision: 0.5000 - Recall: 0.1579 - loss: 0.8281 - val_AUC: 0.8571 - val_Accuracy: 0.7778 - val_Precision: 0.5000 - val_Recall: 0.5000 - val_loss: 0.5743
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - AUC: 0.7456 - Accuracy: 0.6765 - Precision: 0.7500 - Recall: 0.6316 - loss: 0.6137 - val_AUC: 0.9286 - val_Accuracy: 0.6667 - val_Precision: 0.4000 - val_Recall: 1.0000 - val_loss: 0.6309
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - AUC: 0.7228 - Accuracy: 0.6765 - Precision: 0.6818 - Recall: 0.7895 - loss: 0.6050 - val_AUC: 0.8571 - val_Accuracy: 0.5556 - val_Precision: 0.3333 - val_Recall: 1.0000 - val_loss: 0.6332
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - AUC: 0.9772 - Accuracy: 0.8824 - Precision: 0.8261 - Recall: 1.0000 - loss: 0.4001 - val_AUC: 0.8571 - val_Accuracy: 0.5556 - val_Precision: 0.3333 - val_Recall: 1.0000 - val_loss: 0.5950
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51